## Feature engineering : Adding geo data based on location
In this section we'll add location based features using google maps (places API).

## 1. Downloading geodata
First we set up an account in google cloud console, and activated places api

In [ ]:
import os
import pandas as pd 
import googlemaps
from tqdm import tqdm
from geopy.distance import geodesic

API_KEY=open("api").read()

In [ ]:
X_train_0 = pd.read_csv("X_train_J01Z4CN.csv")

In [ ]:
import time
import googlemaps # pip install googlemaps
import pandas as pd # pip install pandas

map_client = googlemaps.Client(API_KEY)

def get_data(id_annonce, location, distance = 5000):
    business_list = []
    response = map_client.places_nearby(
        location=location,
        radius=distance
    )   

    business_list.extend(response.get('results'))
    next_page_token = response.get('next_page_token')

    while next_page_token:
        time.sleep(0.3)
        try :
            response = map_client.places_nearby(
                location=location,
                radius=distance,
                page_token=next_page_token
            )   
            business_list.extend(response.get('results'))
            next_page_token = response.get('next_page_token')
        except Exception as e:
            # print(e)
            break

    df = pd.DataFrame(business_list)
    #df['url'] = 'https://www.google.com/maps/place/?q=place_id:' + df['place_id']
    df['id_annonce']=id_annonce
    return df

In [ ]:
final_results = pd.DataFrame({})
for index, row in tqdm(X_train_0[32908:].iterrows()):
    # if not already_requested(row['id_annonce']) :
            results_per_row = get_data(row['id_annonce'], (row['approximate_latitude'],row['approximate_longitude']))
            final_results=pd.concat([results_per_row, final_results], axis=0)
            final_results.to_pickle("X_train_results_32908_end.pkl")


## 2. Processing the raw geodata

In [36]:
import pandas as pd
import numpy as np
import geopy.distance
from dataprep.eda import plot, plot_missing, plot_correlation, create_report

# Uploading Training and Test set
X_test_0 = pd.read_csv("data/X_test_BEhvxAN.csv")
X_test_0= X_test_0.sort_values(by="id_annonce")

X_train_0 = pd.read_csv("data/X_train_J01Z4CN.csv")
X_train_0 = X_train_0.sort_values(by="id_annonce")
Y_train_0 = pd.read_csv("data/y_train_OXxrJt1.csv")
Y_train_0 =Y_train_0.sort_values(by="id_annonce")

In [34]:
pd.options.display.max_columns=240
pd.options.display.max_rows =240

In [3]:
# Uploading Geodata Test set

In [4]:
X_test=pd.read_pickle("X_test_results.pkl")
X_test = X_test.sort_values(by="id_annonce")

In [5]:
# Uploading Geodata Train set

In [6]:
X_train_1=pd.read_pickle("X_train_results_0_13139.pkl")
X_train_2=pd.read_pickle("X_train_results_13139_18730.pkl")
X_train_3=pd.read_pickle("X_train_results_18730_26338.pkl")
X_train_4=pd.read_pickle("X_train_results_26338_30000.pkl")
X_train_5=pd.read_pickle("X_train_results_30000_32908.pkl")
X_train_6=pd.read_pickle("X_train_results_32908_end.pkl")

X_train=pd.concat([X_train_1, X_train_2, X_train_3, X_train_4, X_train_5, X_train_6])
X_train = X_train.sort_values(by="id_annonce")

In [7]:
def preprocess_geodata(X_test_raw, X_test_geodata):
    
    df =pd.merge(X_test_geodata,X_test_raw, how="left", on =["id_annonce"])
    
    df = df[["id_annonce","approximate_latitude","approximate_longitude", "geometry", "types","user_ratings_total", "rating"]]
    
    df["buis_lat"]= df["geometry"].apply(lambda x: x["location"]["lat"])
    df["buis_lgn"]= df["geometry"].apply(lambda x: x["location"]["lng"])
    
    tf = df.explode("types")
    
    zf = pd.get_dummies(tf, columns=["types"], prefix=[""], prefix_sep="")
    

    print("Calculating number of unique types")
    unique_types = []
    for index, row in X_test_geodata.iterrows():
        types = row["types"]
        unique_types.extend(types)
    unique_types = set(unique_types)
    

    print("Calculating distance...")
    zf["geometry"]=zf.apply(lambda row : 1000*geopy.distance.geodesic((row["buis_lat"],row["buis_lgn"]),(row["approximate_latitude"], row["approximate_longitude"])).km, axis=1)

    print("dropping columns...")
    zf = zf.drop(columns=["approximate_latitude", "approximate_longitude","buis_lat","buis_lgn" ])
    
    zf2= zf.drop(columns=["rating", "user_ratings_total"])

    zf2=zf2.add_prefix("num_")
    
    print("Reformating Dataframe...")
    unique_types = set(unique_types)
    zf[list(unique_types)] = zf[list(unique_types)].mul(np.asarray(zf["geometry"]), axis=0)
    
    zf.replace(to_replace = 0, value = 5000, inplace=True)

    print("Aggregating the results..")
    l= list(unique_types)

    aggr={
    column:'min' for column in l
    }
    aggr['rating']='mean'
    aggr["user_ratings_total"]='sum'

    aggr2={
    "num_"+column:'sum' for column in l
    }
    
    df = zf.groupby('id_annonce').agg(aggr)
    df2= zf2.groupby('num_id_annonce').agg(aggr2)    
        
    return pd.concat([df, df2], axis=1)

In [8]:
X_test_formatted = preprocess_geodata(X_test_0,X_test)

Calculating number of unique types
Calculating distance...
dropping columns...
Reformating Dataframe...
Aggregating the results..


In [24]:
report=create_report(data.reset_index(drop=True), title='Training + Geo Data Report')
report.show_browser()


  0%|                                                 | 0/63523 [00:00<?, ?it/s]

/home/odeck/miniconda3/envs/data-challenge/lib/python3.8/site-packages/dataprep/eda/distribution/render.py:274: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({col: [nrows - npresent]}, index=["Others"]))
/home/odeck/miniconda3/envs/data-challenge/lib/python3.8/site-packages/dataprep/eda/distribution/render.py:274: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({col: [nrows - npresent]}, index=["Others"]))
/home/odeck/miniconda3/envs/data-challenge/lib/python3.8/site-packages/dataprep/eda/distribution/render.py:274: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({col: [nrows - npresent]}, index=["Others"]))
/home/odeck/miniconda3/envs/data-chal

/home/odeck/miniconda3/envs/data-challenge/lib/python3.8/site-packages/dataprep/eda/distribution/render.py:274: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({col: [nrows - npresent]}, index=["Others"]))
/home/odeck/miniconda3/envs/data-challenge/lib/python3.8/site-packages/dataprep/eda/distribution/render.py:274: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({col: [nrows - npresent]}, index=["Others"]))
/home/odeck/miniconda3/envs/data-challenge/lib/python3.8/site-packages/dataprep/eda/distribution/render.py:274: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({col: [nrows - npresent]}, index=["Others"]))
/home/odeck/miniconda3/envs/data-chal

/home/odeck/miniconda3/envs/data-challenge/lib/python3.8/site-packages/dataprep/eda/distribution/render.py:274: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({col: [nrows - npresent]}, index=["Others"]))
/home/odeck/miniconda3/envs/data-challenge/lib/python3.8/site-packages/dataprep/eda/distribution/render.py:274: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({col: [nrows - npresent]}, index=["Others"]))
/home/odeck/miniconda3/envs/data-challenge/lib/python3.8/site-packages/dataprep/eda/distribution/render.py:274: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({col: [nrows - npresent]}, index=["Others"]))
/home/odeck/miniconda3/envs/data-chal

/home/odeck/miniconda3/envs/data-challenge/lib/python3.8/site-packages/dataprep/eda/distribution/render.py:274: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({col: [nrows - npresent]}, index=["Others"]))
/home/odeck/miniconda3/envs/data-challenge/lib/python3.8/site-packages/dataprep/eda/distribution/render.py:274: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({col: [nrows - npresent]}, index=["Others"]))
/home/odeck/miniconda3/envs/data-challenge/lib/python3.8/site-packages/dataprep/eda/distribution/render.py:274: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({col: [nrows - npresent]}, index=["Others"]))
/home/odeck/miniconda3/envs/data-chal

In [35]:
data.corrwith(data["price"]).sort_values()


train_station                 -0.011763
id_annonce                    -0.011623
political                     -0.010517
num_car_dealer                -0.010192
night_club                    -0.010005
transit_station               -0.009661
num_rv_park                   -0.009515
storage                       -0.009508
spa                           -0.008961
gas_station                   -0.008944
num_plumber                   -0.008685
travel_agency                 -0.008210
courthouse                    -0.008159
num_stadium                   -0.008070
num_jewelry_store             -0.007974
health                        -0.007772
num_campground                -0.007485
locality                      -0.007345
pet_store                     -0.007178
zoo                           -0.007166
num_shoe_store                -0.007026
num_meal_takeaway             -0.006852
fire_station                  -0.006830
mosque                        -0.006578
num_city_hall                 -0.006420


In [10]:
X_train_formatted = preprocess_geodata(X_train_0,X_train)

Calculating number of unique types
Calculating distance...
dropping columns...
Reformating Dataframe...
Aggregating the results..


In [29]:
X_train_formatted=X_train_formatted.reset_index()

In [36]:
X_train_formatted

,index,aquarium,spa,hospital,dentist,physiotherapist,primary_school,storage,movie_rental,restaurant,furniture_store,tourist_attraction,jewelry_store,courthouse,cemetery,bowling_alley,home_goods_store,electrician,parking,stadium,gym,veterinary_care,bar,establishment,shopping_mall,airport,hair_care,laundry,post_office,book_store,bank,gas_station,health,car_rental,city_hall,taxi_stand,florist,cafe,rv_park,sublocality_level_1,church,real_estate_agency,painter,food,local_government_office,lodging,library,doctor,pet_store,store,fire_station,electronics_store,convenience_store,hardware_store,night_club,neighborhood,department_store,political,shoe_store,point_of_interest,secondary_school,supermarket,accounting,university,park,meal_takeaway,zoo,beauty_salon,sublocality,art_gallery,car_dealer,liquor_store,embassy,lawyer,school,movie_theater,insurance_agency,campground,car_wash,plumber,funeral_home,meal_delivery,transit_station,moving_company,finance,museum,casino,locality,natural_feature,mosque,place_of_worship,roofing_contractor,drugstore,locksmith,travel_agency,bakery,police,bus_station,grocery_or_supermarket,light_rail_station,amusement_park,car_repair,clothing_store,general_contractor,pharmacy,bicycle_store,premise,train_station,atm,rating,user_ratings_total,num_aquarium,num_spa,num_hospital,num_dentist,num_physiotherapist,num_primary_school,num_storage,num_movie_rental,num_restaurant,num_furniture_store,num_tourist_attraction,num_jewelry_store,num_courthouse,num_cemetery,num_bowling_alley,num_home_goods_store,num_electrician,num_parking,num_stadium,num_gym,num_veterinary_care,num_bar,num_establishment,num_shopping_mall,num_airport,num_hair_care,num_laundry,num_post_office,num_book_store,num_bank,num_gas_station,num_health,num_car_rental,num_city_hall,num_taxi_stand,num_florist,num_cafe,num_rv_park,num_sublocality_level_1,num_church,num_real_estate_agency,num_painter,num_food,num_local_government_office,num_lodging,num_library,num_doctor,num_pet_store,num_store,num_fire_station,num_electronics_store,num_convenience_store,num_hardware_store,num_night_club,num_neighborhood,num_department_store,num_political,num_shoe_store,num_point_of_interest,num_secondary_school,num_supermarket,num_accounting,num_university,num_park,num_meal_takeaway,num_zoo,num_beauty_salon,num_sublocality,num_art_gallery,num_car_dealer,num_liquor_store,num_embassy,num_lawyer,num_school,num_movie_theater,num_insurance_agency,num_campground,num_car_wash,num_plumber,num_funeral_home,num_meal_delivery,num_transit_station,num_moving_company,num_finance,num_museum,num_casino,num_locality,num_natural_feature,num_mosque,num_place_of_worship,num_roofing_contractor,num_drugstore,num_locksmith,num_travel_agency,num_bakery,num_police,num_bus_station,num_grocery_or_supermarket,num_light_rail_station,num_amusement_park,num_car_repair,num_clothing_store,num_general_contractor,num_pharmacy,num_bicycle_store,num_premise,num_train_station,num_atm
0,6774630,5000.0,5000.000000,5000.0,5000.0,5000.0,5000.000000,5000.0,5000.0,643.255006,5000.000000,2014.561216,5000.0,5000.0,5000.0,5000.0,5000.000000,5000.0,3731.373961,5000.00000,5000.000000,5000.0,5000.000000,417.528072,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,1871.399343,5000.000000,1552.543463,5000.0,5000.000000,5000.0,5000.000000,5000.000000,5000.0,5000.0,2014.561216,5000.000000,5000.0,643.255006,5000.000000,417.528072,5000.0,5000.0,5000.000000,576.604271,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,340.310966,5000.0,417.528072,5000.0,5000.000000,5000.0,5000.0,5000.0,5000.000000,5000.0,5000.000000,5000.0,5000.0,3030.869028,5000.000000,5000.0,5000.0,5000.000000,5000.000000,1552.543463,5000.0,5000.000000,5000.0,5000.000000,5000.0,5000.000000,5000.0,1871.399343,5000.0,5000.0,340.310966,5000.0,5000.0,2014.561216,5000.0,5000.0,5000.00000,5000.0,5000.000000,5000.0,5000.0,5000.000000,5000.0,5000.0,3030.869028,576.604271,5000.000000,5000.000000,5000.000000,5000.0,5000.000000,5000.000000,3.892308,18040.0,0,0,0,0,0,0,0,0,3,0,1,0,0,0,0,0,0

## Formatting the results

In [37]:
import pandas as pd
filepath='data/'
pd.options.display.max_rows=140
pd.options.display.max_columns=140

In [53]:
X_train_raw=pd.read_csv(filepath +'X_train_J01Z4CN.csv') 
X_test_raw=pd.read_csv(filepath + 'X_test_BEhvxAN.csv')
Y_train_raw=pd.read_csv(filepath + 'y_train_OXxrJt1.csv')
#places = ["id_annonce", "hospital"]
X_train_geo=pd.read_pickle(filepath+"geodata/X_train_geodata.pkl")
X_test_geo=pd.read_pickle(filepath+"geodata/X_test_geodata.pkl")


In [48]:
X_train_raw[X_train_raw["id_annonce"]==35881331]

,id_annonce,property_type,approximate_latitude,approximate_longitude,city,postal_code,size,floor,land_size,energy_performance_value,energy_performance_category,ghg_value,ghg_category,exposition,nb_rooms,nb_bedrooms,nb_bathrooms,nb_parking_places,nb_boxes,nb_photos,has_a_balcony,nb_terraces,has_a_cellar,has_a_garage,has_air_conditioning,last_floor,upper_floors
421,35881331,appartement,45.783099,4.819287,lyon-4eme,69004,120.0,NaN,NaN,158.0,D,6.0,B,NaN,6.0,5.0,1.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
places = []
# ordering indexes of X_train
X_train_geo = X_train_geo.reset_index()
if len(places)>0:
    X_train_geo = X_train_geo[places]

X_train_geo = X_train_geo.set_index('index')
X_train_geo = X_train_geo.reindex(index=X_train_raw['id_annonce'])
X_train_geo = X_train_geo.reset_index()

# Ordering indexes of X_test
X_test_geo = X_test_geo.reset_index()
if len(places)>0:
    X_test_geo = X_test_geo[places]


X_test_geo = X_test_geo.set_index('index')
X_test_geo = X_test_geo.reindex(index=X_test_raw['id_annonce'])
X_test_geo = X_test_geo.reset_index()


In [33]:
data = pd.concat([X_train_raw, X_test_raw], axis=0)
data_geo=pd.concat([X_train_geo, X_test_geo], axis=0)
data_geo = data_geo.reset_index(drop=True)
data_geo=data_geo.drop(["id_annonce"], axis=1)
data = data.reset_index(drop=True)
final=pd.concat([data, data_geo], axis=1)

In [38]:
final[final["id_annonce"]==6774630]

,id_annonce,property_type,approximate_latitude,approximate_longitude,city,postal_code,size,floor,land_size,energy_performance_value,energy_performance_category,ghg_value,ghg_category,exposition,nb_rooms,nb_bedrooms,nb_bathrooms,nb_parking_places,nb_boxes,nb_photos,has_a_balcony,nb_terraces,has_a_cellar,has_a_garage,has_air_conditioning,last_floor,upper_floors,aquarium,spa,hospital,dentist,physiotherapist,primary_school,storage,movie_rental,restaurant,furniture_store,tourist_attraction,jewelry_store,courthouse,cemetery,bowling_alley,home_goods_store,electrician,parking,stadium,gym,veterinary_care,bar,establishment,shopping_mall,airport,hair_care,laundry,post_office,book_store,bank,gas_station,health,car_rental,city_hall,taxi_stand,florist,cafe,rv_park,sublocality_level_1,church,real_estate_agency,painter,food,...,num_sublocality_level_1,num_church,num_real_estate_agency,num_painter,num_food,num_local_government_office,num_lodging,num_library,num_doctor,num_pet_store,num_store,num_fire_station,num_electronics_store,num_convenience_store,num_hardware_store,num_night_club,num_neighborhood,num_department_store,num_political,num_shoe_store,num_point_of_interest,num_secondary_school,num_supermarket,num_accounting,num_university,num_park,num_meal_takeaway,num_zoo,num_beauty_salon,num_sublocality,num_art_gallery,num_car_dealer,num_liquor_store,num_embassy,num_lawyer,num_school,num_movie_theater,num_insurance_agency,num_campground,num_car_wash,num_plumber,num_funeral_home,num_meal_delivery,num_transit_station,num_moving_company,num_finance,num_museum,num_casino,num_locality,num_natural_feature,num_mosque,num_place_of_worship,num_roofing_contractor,num_drugstore,num_locksmith,num_travel_agency,num_bakery,num_police,num_bus_station,num_grocery_or_supermarket,num_light_rail_station,num_amusement_park,num_car_repair,num_clothing_store,num_general_contractor,num_pharmacy,num_bicycle_store,num_premise,num_train_station,num_atm
12674,6774630,appartement,43.835987,5.786698,manosque,4100,73.0,1.0,NaN,164.0,D,49.0,E,NaN,4.0,3.0,1.0,0.0,0.0,8.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,643.255006,5000.0,2014.561216,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,3731.373961,5000.0,5000.0,5000.0,5000.0,417.528072,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,1871.399343,5000.0,1552.543463,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,2014.561216,5000.0,5000.0,643.255006,...,0,1,0,0,3,0,12,0,0,0,2,0,0,0,0,0,0,0,2,0,18,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2,0,0,0,0,0,0,0,1,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0


In [51]:
final = final.replace([5000], np.nan)


In [52]:
final[final["id_annonce"]==6774630]

,id_annonce,property_type,approximate_latitude,approximate_longitude,city,postal_code,size,floor,land_size,energy_performance_value,energy_performance_category,ghg_value,ghg_category,exposition,nb_rooms,nb_bedrooms,nb_bathrooms,nb_parking_places,nb_boxes,nb_photos,has_a_balcony,nb_terraces,has_a_cellar,has_a_garage,has_air_conditioning,last_floor,upper_floors,aquarium,spa,hospital,dentist,physiotherapist,primary_school,storage,movie_rental,restaurant,furniture_store,tourist_attraction,jewelry_store,courthouse,cemetery,bowling_alley,home_goods_store,electrician,parking,stadium,gym,veterinary_care,bar,establishment,shopping_mall,airport,hair_care,laundry,post_office,book_store,bank,gas_station,health,car_rental,city_hall,taxi_stand,florist,cafe,rv_park,sublocality_level_1,church,real_estate_agency,painter,food,...,num_sublocality_level_1,num_church,num_real_estate_agency,num_painter,num_food,num_local_government_office,num_lodging,num_library,num_doctor,num_pet_store,num_store,num_fire_station,num_electronics_store,num_convenience_store,num_hardware_store,num_night_club,num_neighborhood,num_department_store,num_political,num_shoe_store,num_point_of_interest,num_secondary_school,num_supermarket,num_accounting,num_university,num_park,num_meal_takeaway,num_zoo,num_beauty_salon,num_sublocality,num_art_gallery,num_car_dealer,num_liquor_store,num_embassy,num_lawyer,num_school,num_movie_theater,num_insurance_agency,num_campground,num_car_wash,num_plumber,num_funeral_home,num_meal_delivery,num_transit_station,num_moving_company,num_finance,num_museum,num_casino,num_locality,num_natural_feature,num_mosque,num_place_of_worship,num_roofing_contractor,num_drugstore,num_locksmith,num_travel_agency,num_bakery,num_police,num_bus_station,num_grocery_or_supermarket,num_light_rail_station,num_amusement_park,num_car_repair,num_clothing_store,num_general_contractor,num_pharmacy,num_bicycle_store,num_premise,num_train_station,num_atm
12674,6774630,appartement,43.835987,5.786698,manosque,4100.0,73.0,1.0,NaN,164.0,D,49.0,E,NaN,4.0,3.0,1.0,0.0,0.0,8.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,643.255006,NaN,2014.561216,NaN,NaN,NaN,NaN,NaN,NaN,3731.373961,NaN,NaN,NaN,NaN,417.528072,NaN,NaN,NaN,NaN,NaN,NaN,1871.399343,NaN,1552.543463,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014.561216,NaN,NaN,643.255006,...,0,1,0,0,3,0,12,0,0,0,2,0,0,0,0,0,0,0,2,0,18,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2,0,0,0,0,0,0,0,1,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0


In [62]:
ouf.corrwith(ouf["price"]).sort_values()

id_annonce          -0.011623
price                1.000000
aquarium                  NaN
spa                       NaN
hospital                  NaN
                       ...   
num_pharmacy              NaN
num_bicycle_store         NaN
num_premise               NaN
num_train_station         NaN
num_atm                   NaN
Length: 220, dtype: float64

In [16]:
#X_train_formatted.to_pickle("X_train_geodata.pkl")
#X_test_formatted.to_pickle("X_test_geodata.pkl")